In [2]:
import os
import glob
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [3]:
# --- 0. Định nghĩa các hằng số và thư mục ---
RAW_DATA_DIR = "../data/raw/"
PROCESSED_DATA_DIR = "../data/processed/"

# Chúng ta sẽ cố định độ dài là 4 giây
# Tần số lấy mẫu 22050 Hz * 4 giây = 88200 mẫu
FIXED_LENGTH_SAMPLES = 88200 
# Số lượng Mel bins (chiều cao của ảnh)
N_MELS = 128 

# Ánh xạ nhãn từ tên file (dùng lại từ notebook 02)
emotion_map = {
    "01": "neutral", "02": "calm", "03": "happy", "04": "sad",
    "05": "angry", "06": "fearful", "07": "disgust", "08": "surprised"
}
labels_list = list(emotion_map.values())

# --- 1. Hàm trích xuất Spectrogram có độ dài cố định ---
def extract_spectrogram(file_path, fixed_length):
    try:
        # 1. Tải file âm thanh
        y, sr = librosa.load(file_path, sr=22050)
        
        # 2. Pad (đệm) hoặc Truncate (cắt)
        if len(y) < fixed_length:
            # Đệm file ngắn bằng số 0 (im lặng)
            y = np.pad(y, (0, fixed_length - len(y)), mode='constant')
        else:
            # Cắt file dài
            y = y[:fixed_length]
            
        # 3. Tính Mel Spectrogram
        # Đây là 'hình ảnh' mà CNN sẽ 'nhìn'
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS, n_fft=2048, hop_length=512)
        
        # 4. Chuyển sang thang dB (Log)
        S_db = librosa.power_to_db(S, ref=np.max)
        
        # 5. Lấy nhãn
        filename = os.path.basename(file_path)
        label = emotion_map[filename.split('-')[2]]
        
        return S_db, label
    
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")
        return None, None

# --- 2. Vòng lặp xử lý tất cả file ---
all_spectrograms = []
all_labels = []

print("Bắt đầu trích xuất Spectrogram cho CNN...")

file_paths = glob.glob(os.path.join(RAW_DATA_DIR, "Actor_*", "*.wav"))

for file_path in tqdm(file_paths):
    spectrogram, label = extract_spectrogram(file_path, FIXED_LENGTH_SAMPLES)
    
    if spectrogram is not None:
        all_spectrograms.append(spectrogram)
        all_labels.append(label)

print(f"\nTrích xuất hoàn tất. Xử lý được {len(all_spectrograms)} / {len(file_paths)} file.")

# --- 3. Lưu kết quả ---
X_cnn = np.array(all_spectrograms)

# Mã hóa nhãn (y) bằng LabelEncoder
encoder = LabelEncoder()
encoder.fit(labels_list) # Đảm bảo thứ tự nhất quán
y_cnn = encoder.transform(all_labels)

# In ra shape để kiểm tra
# Shape sẽ là (1440, 128, N) - 1440 ảnh, cao 128, rộng N
# N sẽ khoảng 173 (88200 / 512)
print(f"Mảng đặc trưng CNN (X_cnn) có hình dạng: {X_cnn.shape}")
print(f"Mảng nhãn CNN (y_cnn) có hình dạng: {y_cnn.shape}")

# Lưu vào thư mục processed
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_cnn.npy'), X_cnn)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_cnn.npy'), y_cnn)
np.save(os.path.join(PROCESSED_DATA_DIR, 'cnn_encoder_classes.npy'), encoder.classes_) # Lưu lại tên các lớp

print("Đã lưu X_cnn.npy, y_cnn.npy, và cnn_encoder_classes.npy vào 'data/processed/'")

Bắt đầu trích xuất Spectrogram cho CNN...


  0%|          | 0/1440 [00:00<?, ?it/s]

100%|██████████| 1440/1440 [00:20<00:00, 69.05it/s]



Trích xuất hoàn tất. Xử lý được 1440 / 1440 file.
Mảng đặc trưng CNN (X_cnn) có hình dạng: (1440, 128, 173)
Mảng nhãn CNN (y_cnn) có hình dạng: (1440,)
Đã lưu X_cnn.npy, y_cnn.npy, và cnn_encoder_classes.npy vào 'data/processed/'


In [4]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# --- 1. Tải dữ liệu đã xử lý cho CNN ---
PROCESSED_DATA_DIR = "../data/processed/"

X = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_cnn.npy'))
y = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_cnn.npy'))
class_names = np.load(os.path.join(PROCESSED_DATA_DIR, 'cnn_encoder_classes.npy'), allow_pickle=True)

print(f"Đã tải X với shape: {X.shape}")
print(f"Đã tải y với shape: {y.shape}")
print(f"Các lớp: {class_names}")

# --- 2. Chuẩn bị dữ liệu cho Keras ---

# a) Reshape X để thêm "kênh" (channel)
# CNN yêu cầu đầu vào có 4 chiều: (batch_size, height, width, channels)
# Vì spectrogram là ảnh xám, chúng ta có 1 kênh (channel=1)
X_cnn_reshaped = X[..., np.newaxis] # Thêm một chiều ở cuối
print(f"Shape của X sau khi reshape: {X_cnn_reshaped.shape}")

# b) Chuyển y sang dạng One-Hot Encoding
# Ví dụ: số '2' (happy) -> [0, 0, 1, 0, 0, 0, 0, 0]
num_classes = len(class_names)
y_cnn_categorical = to_categorical(y, num_classes=num_classes)
print(f"Shape của y sau khi one-hot: {y_cnn_categorical.shape}")


# --- 3. Chia dữ liệu (Train/Test Split) ---
# Chia 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn_reshaped, 
    y_cnn_categorical, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_cnn_categorical
)

print(f"Shape X_train: {X_train.shape}")
print(f"Shape X_test: {X_test.shape}")


# --- 4. Định nghĩa Kiến trúc Mô hình CNN ---

# Lấy kích thước input từ shape của X_train
input_shape = X_train.shape[1:] # Sẽ là (128, 173, 1)

model_cnn = Sequential([
    # Lớp Tích chập (Convolutional) đầu tiên
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(), # Giúp huấn luyện ổn định hơn
    
    # Lớp Tích chập thứ hai
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.25), # Dropout 25% nơ-ron để tránh overfitting

    # Lớp Tích chập thứ ba
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.25),

    # Làm phẳng (Flatten) dữ liệu từ 3D (ảnh) thành 1D (vector)
    Flatten(),
    
    # Lớp kết nối đầy đủ (Dense)
    Dense(512, activation='relu'),
    Dropout(0.5), # Dropout 50%
    
    # Lớp Output
    # Dùng 'softmax' cho bài toán phân loại nhiều lớp
    Dense(num_classes, activation='softmax') 
])

# --- 5. Biên dịch (Compile) Mô hình ---
model_cnn.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', # Dùng loss này vì y đã được one-hot
    metrics=['accuracy']
)

# In ra tóm tắt kiến trúc mô hình
model_cnn.summary()

Đã tải X với shape: (1440, 128, 173)
Đã tải y với shape: (1440,)
Các lớp: ['angry' 'calm' 'disgust' 'fearful' 'happy' 'neutral' 'sad' 'surprised']
Shape của X sau khi reshape: (1440, 128, 173, 1)
Shape của y sau khi one-hot: (1440, 8)
Shape X_train: (1152, 128, 173, 1)
Shape X_test: (288, 128, 173, 1)


e:\FSB\DSP\DSP501_SpeechEmotionRecognition\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 171, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 85, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 63, 85, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 83, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 41, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 41, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 41, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 39, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 19, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 19, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 19, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 34048)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    17,433,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         4,104 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,530,760 (66.87 MB)

 Trainable params: 17,530,312 (66.87 MB)

 Non-trainable params: 448 (1.75 KB)

In [4]:
# --- 6. Huấn luyện (Fit) Mô hình ---

# epochs=50: Huấn luyện lặp đi lặp lại 50 lần qua toàn bộ dữ liệu
# batch_size=32: Mỗi lần tính toán, mô hình sẽ "nhìn" 32 spectrogram
# validation_data: Dùng tập X_test, y_test để kiểm tra hiệu suất
#                  sau mỗi epoch trên dữ liệu mà nó chưa thấy
print("Bắt đầu huấn luyện mô hình CNN... (Quá trình này sẽ mất nhiều thời gian trên CPU)")

history = model_cnn.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1 # In ra thanh tiến trình
)

print("Đã huấn luyện xong!")

# --- 7. Đánh giá cuối cùng trên Tập Test ---
print("\nĐánh giá hiệu suất cuối cùng trên tập Test:")
test_loss, test_accuracy = model_cnn.evaluate(X_test, y_test, verbose=0)

print(f"Loss trên tập Test: {test_loss:.4f}")
print(f"Độ chính xác (Accuracy) trên tập Test: {test_accuracy * 100:.2f}%")

Bắt đầu huấn luyện mô hình CNN... (Quá trình này sẽ mất nhiều thời gian trên CPU)
Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 678ms/step - accuracy: 0.2318 - loss: 6.0407 - val_accuracy: 0.1319 - val_loss: 197.6679
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 28s 767ms/step - accuracy: 0.2639 - loss: 1.9195 - val_accuracy: 0.1215 - val_loss: 58.6504
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 24s 675ms/step - accuracy: 0.2899 - loss: 1.8393 - val_accuracy: 0.1597 - val_loss: 19.0918
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 26s 719ms/step - accuracy: 0.3160 - loss: 1.7698 - val_accuracy: 0.1840 - val_loss: 12.4128
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 689ms/step - accuracy: 0.3394 - loss: 1.7160 - val_accuracy: 0.1458 - val_loss: 2.6786
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 26s 727ms/step - accuracy: 0.3420 - loss: 1.6648 - val_accuracy: 0.3854 - val_loss: 1.8078
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 731ms/step - accuracy: 0.3924 - loss: 1.5718 - val_accuracy: 0.3646 - val_loss: 1.7793
Epoch 8/50

In [5]:
import matplotlib.pyplot as plt

# --- 8. Vẽ biểu đồ Lịch sử Huấn luyện ---

print("Đang vẽ biểu đồ Accuracy và Loss...")

history_dict = history.history

# Lấy các giá trị
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs_range = range(1, len(acc) + 1)

# --- Biểu đồ 1: Accuracy (Độ chính xác) ---
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1) # 1 hàng, 2 cột, plot thứ 1
plt.plot(epochs_range, acc, 'bo-', label='Training Accuracy')
plt.plot(epochs_range, val_acc, 'ro-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# --- Biểu đồ 2: Loss (Độ lỗi) ---
plt.subplot(1, 2, 2) # 1 hàng, 2 cột, plot thứ 2
plt.plot(epochs_range, loss, 'bo-', label='Training Loss')
plt.plot(epochs_range, val_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Đang vẽ biểu đồ Accuracy và Loss...


NameError: name 'history' is not defined

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# --- 1. Định nghĩa lại Kiến trúc Mô hình (Để reset weights) ---
# (Chúng ta dùng lại y hệt kiến trúc cũ)

num_classes = y_train.shape[1] # Phải là 8
input_shape = X_train.shape[1:] # Phải là (128, 173, 1)

model_cnn = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax') 
])

# --- 2. Biên dịch (Compile) Mô hình ---
model_cnn.compile(
    optimizer='adam', 
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- 3. Định nghĩa Callback "Early Stopping" ---
early_stopping_callback = EarlyStopping(
    monitor='val_loss',     # Theo dõi độ lỗi trên tập test
    patience=5,             # Dừng nếu 5 epochs liên tiếp không cải thiện
    verbose=1,              # In thông báo khi dừng
    restore_best_weights=True # Tự động khôi phục mô hình tốt nhất
)

# --- 4. Huấn luyện (Fit) Mô hình với Callback ---
print("Bắt đầu huấn luyện CNN với Early Stopping...")

history_es = model_cnn.fit(
    X_train, y_train,
    epochs=50,                # Chúng ta vẫn đặt 50, nhưng nó sẽ tự dừng sớm
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping_callback] # Thêm callback vào đây
)

print("Đã huấn luyện xong!")

# --- 5. Đánh giá cuối cùng trên Tập Test (với mô hình tốt nhất) ---
print("\nĐánh giá hiệu suất (mô hình tốt nhất đã được khôi phục):")
test_loss, test_accuracy = model_cnn.evaluate(X_test, y_test, verbose=0)

print(f"Loss trên tập Test: {test_loss:.4f}")
print(f"Độ chính xác (Accuracy) trên tập Test: {test_accuracy * 100:.2f}%")

Bắt đầu huấn luyện CNN với Early Stopping...
Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 755ms/step - accuracy: 0.2188 - loss: 6.6427 - val_accuracy: 0.1319 - val_loss: 158.0466
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 28s 778ms/step - accuracy: 0.2561 - loss: 1.9525 - val_accuracy: 0.1319 - val_loss: 87.3600
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 23s 636ms/step - accuracy: 0.2908 - loss: 1.8855 - val_accuracy: 0.0972 - val_loss: 23.0469
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 592ms/step - accuracy: 0.3359 - loss: 1.7608 - val_accuracy: 0.2153 - val_loss: 2.1503
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 22s 605ms/step - accuracy: 0.3273 - loss: 1.7395 - val_accuracy: 0.2118 - val_loss: 2.2372
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 586ms/step - accuracy: 0.3333 - loss: 1.6468 - val_accuracy: 0.3090 - val_loss: 1.8037
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 581ms/step - accuracy: 0.3637 - loss: 1.5919 - val_accuracy: 0.3472 - val_loss: 1.8243
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 588ms/